# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.129985e+02     1.637541e+00
 * time: 0.0956120491027832
     1     1.115662e+01     9.410288e-01
 * time: 1.0070140361785889
     2    -1.160252e+01     9.780758e-01
 * time: 1.1582140922546387
     3    -3.402800e+01     7.866685e-01
 * time: 1.3265349864959717
     4    -4.775236e+01     5.496904e-01
 * time: 1.4821901321411133
     5    -5.689252e+01     2.114102e-01
 * time: 1.630270004272461
     6    -5.970779e+01     1.485947e-01
 * time: 1.741314172744751
     7    -6.083787e+01     7.109999e-02
 * time: 1.8592400550842285
     8    -6.128728e+01     6.783320e-02
 * time: 1.9723501205444336
     9    -6.157791e+01     5.045313e-02
 * time: 2.0906009674072266
    10    -6.179258e+01     2.754382e-02
 * time: 2.2092061042785645
    11    -6.196370e+01     2.016929e-02
 * time: 2.3208811283111572
    12    -6.203990e+01     1.737780e-02
 * time: 2.439578056335449
    13    -6.211952e+01     1.523035e-02
 * time: 2.5499739646911

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557730
    AtomicNonlocal      14.8522689
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336823

    total               -62.261666459758
